Лабораторная работа №7

1. Загрузите объекты из новостного датасета 20 newsgroups, относящиеся к категориям "космос"и "атеизм"(инструкция приведена выше).

In [1]:
import pandas
import numpy
from sklearn import datasets 
from sklearn.model_selection import train_test_split

newsgroup = datasets.fetch_20newsgroups(subset='all', categories=['alt.atheism', 'sci.space'])


2. Вычислите TF-IDF-признаки для всех текстов. Обратите внимание, что в этом задании мы предлагаем вам вычислить TF-IDF по
всем данным. При таком подходе получается, что признаки на обучающем множестве используют информацию из тестовой выборки
3
— но такая ситуация вполне законна, поскольку мы не используем значения целевой переменной из теста. На практике нередко
встречаются ситуации, когда признаки объектов тестовой выборки
известны на момент обучения, и поэтому можно ими пользоваться
при обучении алгоритма.

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.svm import SVC 
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

clf = TfidfVectorizer()
clf.fit_transform(newsgroup.data)



<1786x28382 sparse matrix of type '<class 'numpy.float64'>'
	with 303138 stored elements in Compressed Sparse Row format>

3. Подберите минимальный лучший параметр C из множества [10−5
, 10−4
, ...104
, 105
]
для SVM с линейным ядром (kernel=’linear’) при помощи кроссвалидации по 5 блокам. Укажите параметр random_state=241 и
для SVM, и для KFold. В качестве меры качества используйте долю верных ответов (accuracy).

In [3]:
grid = {'C': numpy.power(10.0 ,numpy.arange(-5, 6))}
cv = KFold(n_splits = 5, shuffle = True, random_state = 241)
sv = SVC(kernel='linear', random_state=241)
gs = GridSearchCV(sv, grid, scoring = 'accuracy', cv = cv)
gs.fit(clf.transform(newsgroup.data), newsgroup.target)

GridSearchCV(cv=KFold(n_splits=5, random_state=241, shuffle=True),
             estimator=SVC(kernel='linear', random_state=241),
             param_grid={'C': array([1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02,
       1.e+03, 1.e+04, 1.e+05])},
             scoring='accuracy')

4. Обучите SVM по всей выборке с лучшим параметром C, найденным
на предыдущем шаге

In [4]:
best_c = gs.best_params_['C']
model = SVC(C = best_c, kernel='linear', random_state=241)
model.fit(clf.transform(newsgroup.data), newsgroup.target)


SVC(kernel='linear', random_state=241)

5. Найдите 10 слов с наибольшим по модулю весом. Они являются
ответом на это задание. Укажите их через запятую, в нижнем регистре, в лексикографическом порядке.


In [5]:
words = clf.get_feature_names_out()
coef = pandas.DataFrame(model.coef_.data, model.coef_.indices)
top_words = coef[0].map(lambda w: abs(w)).sort_values(ascending=False).head(10).index.map(lambda i: words[i])
numpy.sort(top_words)

array(['atheism', 'atheists', 'bible', 'god', 'keith', 'moon', 'religion',
       'sci', 'sky', 'space'], dtype=object)